<div class='alert alert-warning'>

SciPy's interactive examples with Jupyterlite are experimental and may not always work as expected. Execution of cells containing imports may result in large downloads (up to 60MB of content for the first import from SciPy). Load times when importing from SciPy may take roughly 10-20 seconds. If you notice any problems, feel free to open an [issue](https://github.com/scipy/scipy/issues/new/choose).

</div>

Evaluate the derivative of ``np.exp`` at several points ``x``.


In [ ]:
import numpy as np
from scipy.differentiate import derivative
f = np.exp
df = np.exp  # true derivative
x = np.linspace(1, 2, 5)
res = derivative(f, x)
res.df  # approximation of the derivative

array([2.71828183, 3.49034296, 4.48168907, 5.75460268, 7.3890561 ])

In [ ]:
res.error  # estimate of the error

array([7.13740178e-12, 9.16600129e-12, 1.17594823e-11, 1.51061386e-11,
       1.94262384e-11])

In [ ]:
abs(res.df - df(x))  # true error

array([2.53130850e-14, 3.55271368e-14, 5.77315973e-14, 5.59552404e-14,
       6.92779167e-14])

Show the convergence of the approximation as the step size is reduced.
Each iteration, the step size is reduced by `step_factor`, so for
sufficiently small initial step, each iteration reduces the error by a
factor of ``1/step_factor**order`` until finite precision arithmetic
inhibits further improvement.


In [ ]:
import matplotlib.pyplot as plt
iter = list(range(1, 12))  # maximum iterations
hfac = 2  # step size reduction per iteration
hdir = [-1, 0, 1]  # compare left-, central-, and right- steps
order = 4  # order of differentiation formula
x = 1
ref = df(x)
errors = []  # true error
for i in iter:
    res = derivative(f, x, maxiter=i, step_factor=hfac,
                     step_direction=hdir, order=order,
                     # prevent early termination
                     tolerances=dict(atol=0, rtol=0))
    errors.append(abs(res.df - ref))
errors = np.array(errors)
plt.semilogy(iter, errors[:, 0], label='left differences')
plt.semilogy(iter, errors[:, 1], label='central differences')
plt.semilogy(iter, errors[:, 2], label='right differences')
plt.xlabel('iteration')
plt.ylabel('error')
plt.legend()
plt.show()

In [ ]:
(errors[1, 1] / errors[0, 1], 1 / hfac**order)

(0.06215223140159822, 0.0625)

The implementation is vectorized over `x`, `step_direction`, and `args`.
The function is evaluated once before the first iteration to perform input
validation and standardization, and once per iteration thereafter.


In [ ]:
def f(x, p):
    f.nit += 1
    return x**p
f.nit = 0
def df(x, p):
    return p*x**(p-1)
x = np.arange(1, 5)
p = np.arange(1, 6).reshape((-1, 1))
hdir = np.arange(-1, 2).reshape((-1, 1, 1))
res = derivative(f, x, args=(p,), step_direction=hdir, maxiter=1)
np.allclose(res.df, df(x, p))

True

In [ ]:
res.df.shape

(3, 5, 4)

In [ ]:
f.nit

2

By default, `preserve_shape` is False, and therefore the callable
`f` may be called with arrays of any broadcastable shapes.
For example:


In [ ]:
shapes = []
def f(x, c):
   shape = np.broadcast_shapes(x.shape, c.shape)
   shapes.append(shape)
   return np.sin(c*x)

c = [1, 5, 10, 20]
res = derivative(f, 0, args=(c,))
shapes

[(4,), (4, 8), (4, 2), (3, 2), (2, 2), (1, 2)]

To understand where these shapes are coming from - and to better
understand how `derivative` computes accurate results - note that
higher values of ``c`` correspond with higher frequency sinusoids.
The higher frequency sinusoids make the function's derivative change
faster, so more function evaluations are required to achieve the target
accuracy:


In [ ]:
res.nfev

array([11, 13, 15, 17], dtype=int32)

The initial ``shape``, ``(4,)``, corresponds with evaluating the
function at a single abscissa and all four frequencies; this is used
for input validation and to determine the size and dtype of the arrays
that store results. The next shape corresponds with evaluating the
function at an initial grid of abscissae and all four frequencies.
Successive calls to the function evaluate the function at two more
abscissae, increasing the effective order of the approximation by two.
However, in later function evaluations, the function is evaluated at
fewer frequencies because the corresponding derivative has already
converged to the required tolerance. This saves function evaluations to
improve performance, but it requires the function to accept arguments of
any shape.

"Vector-valued" functions are unlikely to satisfy this requirement.
For example, consider


In [ ]:
def f(x):
   return [x, np.sin(3*x), x+np.sin(10*x), np.sin(20*x)*(x-1)**2]

This integrand is not compatible with `derivative` as written; for instance,
the shape of the output will not be the same as the shape of ``x``. Such a
function *could* be converted to a compatible form with the introduction of
additional parameters, but this would be inconvenient. In such cases,
a simpler solution would be to use `preserve_shape`.


In [ ]:
shapes = []
def f(x):
    shapes.append(x.shape)
    x0, x1, x2, x3 = x
    return [x0, np.sin(3*x1), x2+np.sin(10*x2), np.sin(20*x3)*(x3-1)**2]

x = np.zeros(4)
res = derivative(f, x, preserve_shape=True)
shapes

[(4,), (4, 8), (4, 2), (4, 2), (4, 2), (4, 2)]

Here, the shape of ``x`` is ``(4,)``. With ``preserve_shape=True``, the
function may be called with argument ``x`` of shape ``(4,)`` or ``(4, n)``,
and this is what we observe.
